Nombres:
-David del Cerro Domínguez
-Sergio Ramos Mesa

Ejercicio 2

 En el siguiente puzle infantil se pide rellenar los espacios en blanco con  números  del  1  al  9 de  tal  
 modo que la ecuación sea  correcta  siguiendo  el  orden de las operaciones. Los números no pueden repetirse.
 Es decir, en cada uno  de  los  9  huecos  en  blancos  deberá  ir  un  número  sin  repetirse.
 Según declaraciones   del   profesor   Tran Phuong existen   362.880   combinaciones   posibles para colocar los
 9 dígitos en los 9 espacios en blanco, pero sólo unas pocas son correctas

Para resolver este problema utilizamos la búsqueda en profundidad, ya que necesitamos colocar los números en todos los huecos para obtener una solución, por lo tanto nos resulta más eficaz este método. La complejidad está directamente relacionada con el factor de ramificación, que en este caso va disminuyendo según avanzamos en profundidad. 

In [27]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [57]:
class Puzzle(Problem):
    operations = ()
    def __init__(self, initial, operations, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.operations = operations
        self.initial = initial
        self.goal = goal
            
    def actions(self,estado):
        accs = list()
        digsFaltan = estado[1]
        datos = estado[0]
        if 0 in datos:
            pos = datos.index(0)

            if pos > 0 and self.operations[pos - 1] == ":":
                i = 0
                while i < len(digsFaltan):
                    if(estado[2] % digsFaltan[i] == 0):
                        accs.append(digsFaltan[i])
                    i = i + 1
            else: #En caso de división, solo cogeremos de los digitos que faltan aquellos que son divisores del resultado.
                i = 0
                while i < len(digsFaltan):
                        accs.append(digsFaltan[i])
                        i = i + 1
        
        return accs

    def result(self,estado,accion):
        #Transformamos nuestros datos a list para así operar con ellos con mayor facilidad
        digs = list(estado[1]) 
        digs.remove(accion)  
        nuevoEstado1 = tuple(digs)
        datos = list(estado[0])
        pos = estado[0].index(0)
        datos[pos] = accion
        nuevoEstado0 = tuple(datos)
        if(estado[0].index(0) == 0): #Si el dato a procesar es el primero, el resultado es igual al dato
            resul = accion
        else:     
            #Añadimos a resul el numero que acabamos de añadir
                if(self.operations[estado[0].index(0) - 1] == "+"):
                    resul = estado[2] + accion
                if(self.operations[estado[0].index(0) - 1] == "-"):
                    resul = estado[2] - accion
                if(self.operations[estado[0].index(0) - 1] == "x"):
                    resul = estado[2] * accion
                if(self.operations[estado[0].index(0) - 1] == ":"):
                    resul = estado[2] // accion
        
         #Comprobamos que el numero siguiente si es un dato del problema, se incluya
        if(pos + 1 < len(datos) and datos[pos + 1] > 9):
            if(self.operations[pos] == "+"):
                resul = resul + datos[pos + 1]
            if(self.operations[pos] == "-"):
                resul = resul - datos[pos + 1]
            if(self.operations[pos] == "x"):
                resul = resul * datos[pos + 1]
            if(self.operations[pos] == ":"):
                resul = resul // datos[pos + 1]

        return (nuevoEstado0,nuevoEstado1,resul)
        
    
        
    def goal_test(self,estado):
        return (self.goal == estado[2])

In [6]:
from search import *

In [58]:
#Creamos el estado inicial, siendo 0 los huecos, la segunda componente los digitos que faltan, es decir todos.
estado_inicial = ((0,13,0,0,0,12,0,0,11,0,0,0,10),(1,2,3,4,5,6,7,8,9),0) 
depth_first_tree_search(Puzzle(estado_inicial,("+","x",":","+","+","x","-","-","+","x",":","-"),66)).solution()

[9, 7, 2, 8, 4, 3, 6, 1, 5]